![LangChain](img/langchain.jpeg)

**LangChain** est un framework open source conçu pour construire des applications d’intelligence artificielle autour des modèles de langage (LLMs) comme GPT, Claude ou Mistral. Il a la capacité de se connecter aux LLMs, à des sources de données, des outils, des chaînes de raisonnement et des moyens de stockage pour créer des systèmes interactifs et dynamiques.

![LangChain components](img/langchain_components.png)

> **LLM**
--
Les LLMs sont les moteurs de raisonnement, de génération de texte ou de prise de décision. LangChain les encapsule pour les intégrer facilement dans des workflows intelligents.

> **Prompts**
--
Les prompts sont la manière dont on guide un modèle. LangChain fournit des outils pour construire des prompts dynamiques, réutilisables et paramétrables.

> **Chains**
--
Une `chain` est une séquence logique d’appels à un LLM et à d’autres composants (par exemple : extraction d’information → recherche vectorielle → génération de réponse). Elle permet de créer des **pipelines IA personnalisés** pour des tâches complexes.

> **Memory**
--
LangChain permet de gérer une mémoire conversationnelle, c’est-à-dire la capacité à se souvenir des échanges passés. Cela rend les interactions plus naturelles et contextuelles dans les agents ou les chatbots.

> **Agents**
--
Les agents vont plus loin : ils choisissent dynamiquement les actions à effectuer à partir d’outils disponibles (recherche web, calcul, consultation de base de données…). Ils peuvent décider quel outil appeler, avec quelles données, et enchaîner plusieurs étapes de façon autonome.

> **Documents Loader**, **Text Splitters**, **Indexes** et **Vector DB**
--
> Ces composants forment la chaîne d’ingestion de connaissances :
> - le Documents Loader charge des documents bruts depuis des fichiers, APIs, bases de données ou sites web.
> - les Text Splitters découpent ces documents en chunks (morceaux de texte) pour respecter les limites de contexte des LLMs.
> - le Vector DB : encode les chunks en vecteurs (via des embeddings) et les stocke dans une base vectorielle pour permettre une recherche par similarité.
> - les Indexes centralisent et organisent ces composants pour structurer une base consultable. Ils permettent à un agent ou une chaîne de retrouver les informations pertinentes pour une tâche donnée (Q/R, résumé, etc.).

# 1. Chargement du modèle
___

## LLM local :

Dans cette section, nous chargeons un modèle de langage local grâce à **Ollama**. Cela permet de travailler avec un **LLM directement sur notre machine**, sans connexion à une API externe.

Nous utilisons ici la classe `ChatOllama` de **LangChain**, qui nous permet d’interagir facilement avec un modèle comme llama3 déjà téléchargé via Ollama.

GTP-OSS:20b, Mistral-Small3.2 24B GLM 4.7 Flash

## LLM Cloud :
Mistral

In [1]:
import os
from IPython.display import display, Markdown
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_mistralai.chat_models import ChatMistralAI

# Chargement des clés d'API se trouvant dans le fichier .env.
# Ceci permet d'utiliser des modèles en ligne comme gpt-x, deepseek-x, etc...
load_dotenv(override=True)

model = ChatOllama(model="glm-4.7-flash")

#model = ChatMistralAI(model="mistral-large-latest", api_key=os.getenv("MISTRAL_API_KEY"))

# 2. Requête basique
___

Maintenant que notre modèle est chargé, nous pouvons lui envoyer une première requête simple. Ici, nous utilisons la méthode `.invoke()` pour poser une question directe.

Cela nous permet de tester rapidement le bon fonctionnement du modèle et d’observer comment il formule ses réponses.

In [2]:
# Envoie une requête simple au modèle LLM via la méthode `invoke`
# Ici, on pose un problème de mathématiques en langage naturel
result = model.invoke("Résous ce problème de mathématiques. Quel est le résultat de la division de 4 par 2 ?")

# Affiche uniquement la réponse générée par le modèle (sans métadonnées)
display(Markdown(result.content))

Le résultat est **2**.

En mathématiques, $4 \div 2 = 2$.

### 🧩 Exercices

#### Exercice 1

Utilisez le modèle pour transformer une phrase simple en la réécrivant dans un style littéraire spécifique.
1.	Envoyez une requête directe (sans PromptTemplate) via .invoke() contenant :
- une instruction claire au modèle,
- une phrase source,
- le style souhaité (ex. : Shakespeare, roman noir, science-fiction, etc.).
2.	Affichez uniquement le résultat retourné par le LLM.

In [3]:
# Votre code ici

#### Exercice 2

Tu es en mission pour rédiger un message diplomatique adressé à une civilisation extraterrestre très susceptible.
1.	Envoyez une requête au modèle via .invoke() avec un prompt complet :
- contexte fictif : situation tendue,
- contraintes : éviter certains mots, rester poli,
- objectif : obtenir la paix ou proposer une alliance.
2.	Observez comment le modèle gère le ton et les instructions.

In [4]:
# Votre code ici

# 3. Conversations avec le modèle
___

Même si tout mettre dans un seul message peut fonctionner dans des cas simples, des types de messages différent nous donne plus de contrôle sur le dialogue et permet de mieux exploiter les capacités du modèle, surtout dans des systèmes plus complexes comme des agents ou des chatbots.

C'est pour cela que, plutôt que de tout écrire dans une seule phrase, il est recommandé de distinguer différents types de messages :
- `SystemMessage` : permet de définir le rôle ou le comportement attendu du modèle (par exemple : “Vous êtes un assistant qui répond en français”).
- `HumanMessage` : correspond à ce que vous demandez réellement au modèle.
- `AIMessage` : représente une réponse précédente du modèle, utile si nous construisons une conversation continue.

In [5]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

### 3.1 Conversation sans mémoire (stateless)

Dans l'exemple suivant, nous structurons notre requête en simulant une interaction avec le modèle.
Nous séparons le contexte général (via un SystemMessage) de la question posée (via un HumanMessage).

In [6]:
# On définit une liste de messages structurés pour guider le comportement du modèle
messages = [
    # Le SystemMessage précise le rôle ou l'objectif général : ici, résoudre un problème mathématique
    SystemMessage(content="Résous ce problème de mathématiques"),

    # Le HumanMessage contient la question concrète posée par l'utilisateur
    HumanMessage(content="Quel est le résultat de la division de 4 par 2 ?")
]

result = model.invoke(messages)

display(Markdown(result.content))

Le résultat de la division de 4 par 2 est **2**.

### 3.2 Conversation avec mémoire (stateful)

Dans l'exemple qui suit, nous simulons une conversation à plusieurs tours avec le modèle.
Nous utilisons un AIMessage pour rappeler la réponse précédente, ce qui permet au modèle de garder le fil du dialogue et de répondre naturellement à une nouvelle question en lien avec la précédente.

In [7]:
# On construit une liste de messages simulant une conversation en plusieurs étapes
messages = [
    # Le SystemMessage définit le rôle général du modèle : ici, résoudre des problèmes de mathématiques
    SystemMessage(content="Résous ce problème de mathématiques"),

    # Premier message de l'utilisateur : une question simple
    HumanMessage(content="Quel est le résultat de la division de 4 par 2 ?"),

    # Réponse simulée du modèle à la première question (permet de maintenir le contexte)
    AIMessage(content="Le résultat de la division de 4 par 2 est égal à 2."),

    # Deuxième question de l'utilisateur, liée à la précédente
    HumanMessage(content="Et 8 multiplié par 4 ?"),
]

result = model.invoke(messages)

display(Markdown(result.content))

Le résultat de $8$ multiplié par $4$ est égal à $32$.

Dans le second exemple ci-dessous, nous mettons en place une boucle de conversation interactive avec le modèle.
À chaque échange, la question de l’utilisateur et la réponse du modèle sont ajoutées à l’historique (`chat_history`).
Cela permet au LLM de garder en mémoire le contexte et de répondre de façon plus cohérente tout au long de la discussion.

In [8]:
# Initialisation de l'historique des messages
chat_history = []

# Message système : donne un rôle au modèle pour toute la session
system_message = SystemMessage(content="Tu es un expert en mathématiques et un pédagogue dans ce domaine.")
chat_history.append(system_message)


# Boucle principale de conversation (s'arrête si l'utilisateur tape 'exit')
# ⚠️ `while Flase: ` à modier en `while True: ` et inversément lorsque vous souhaitez désactiver ou activer cet exemple
while True:
    query = input("Vous : ")
    if query.lower() == "exit":
        break  # Sortie de la boucle

    # Ajout de la question de l'utilisateur dans l'historique
    chat_history.append(HumanMessage(content=query))

    # Envoi de tout l'historique au modèle pour maintenir le contexte
    result = model.invoke(chat_history)
    response = result.content

    # Ajout de la réponse du modèle dans l'historique
    chat_history.append(AIMessage(content=response))

    # Affichage de la réponse à l'utilisateur
    print(response)


print("------ Historique des messages ------")
print(chat_history)

Le résultat est **11**.

Voici une petite explication pour visualiser l'opération :

1.  Si tu mets tes 5 doigts d'une main et que tu en ajoutes 6 autres, cela fait 11 doigts au total.
2.  Tu peux aussi le voir comme : $5 + 5 = 10$, et il te reste encore $1$ à ajouter.
------ Historique des messages ------
[SystemMessage(content='Tu es un expert en mathématiques et un pédagogue dans ce domaine.', additional_kwargs={}, response_metadata={}), HumanMessage(content='5+6', additional_kwargs={}, response_metadata={}), AIMessage(content="Le résultat est **11**.\n\nVoici une petite explication pour visualiser l'opération :\n\n1.  Si tu mets tes 5 doigts d'une main et que tu en ajoutes 6 autres, cela fait 11 doigts au total.\n2.  Tu peux aussi le voir comme : $5 + 5 = 10$, et il te reste encore $1$ à ajouter.", additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


### 🧩 Exercices

#### Exercice 3

1.	Créez une liste messages avec :
- un SystemMessage qui indique que l’IA est un expert dans un domaine de ton choix (maths, histoire, cinéma, etc.),
- un HumanMessage qui pose une question à l’IA.
2.	Envoyez cette liste à model.invoke(messages) et affiche la réponse.

In [9]:
# Votre code ici

#### Exercice 4

Créer une mini-conversation avec l’IA, où chaque question/réponse est ajoutée à l’historique des messages. L’IA doit se souvenir de l’échange précédent.

1.	Initialisez une liste messages avec un SystemMessage définissant le rôle de l’IA.
2.	Dans une boucle :
- Demandez une question à l’utilisateur (input()),
- Ajoutez un HumanMessage à la liste,
- Envoyez la liste complète à model.invoke(...),
- Affichez la réponse de l’IA,
- Ajoutez cette réponse comme AIMessage à la liste.
3.	Arrêtez la boucle si l’utilisateur entre "stop".

In [10]:
# Votre code ici

# 4. Conversations avec le modèle à l'aide de Prompt Templates
___

Nous allons explorer l’utilisation de `ChatPromptTemplate`, un outil qui permet de structurer proprement les messages envoyés à un modèle de type “chat” (comme GPT-4).

`ChatPromptTemplate` permet de construire une conversation multi-rôle en distinguant les messages système (règles, rôle de l’IA), humains (questions ou commandes) et les réponses de l’IA.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

### 4.1 Prompt conversation à rôle unique (human)

Ce type de prompt utilise la fonction `.from_template( )` et est de type `human` par défaut, c'est un prompte simple "tout en un" où il n'est pas possible de contrôler le rôle.

In [12]:
# Définition d'un template simple (texte brut avec variables), sans rôles explicites
template = "Tu es un expert en mathématiques et un pédagogue dans ce domaine. Calcule le double de {value_1}, puis celui de {value_2}"

# Création du prompt à partir du template ; ce sera un message unique de type 'human' par défaut
chat_prompt_template = ChatPromptTemplate.from_template(template)

# Injection des valeurs dans les variables du template
prompt_value = chat_prompt_template.invoke({"value_1": 12, "value_2": 34})

# Envoi du prompt au modèle pour obtenir une réponse
result = model.invoke(prompt_value)

display(Markdown(result.content))

Bonjour ! C'est un excellent exercice pour s'entraîner avec les multiplications.

Pour trouver le double d'un nombre, on le multiplie simplement par 2 (ou on l'ajoute à lui-même).

Voici les calculs étape par étape :

1.  **Le double de 12 :**
    $12 \times 2 = \mathbf{24}$

2.  **Le double de 34 :**
    $34 \times 2 = \mathbf{68}$

### 4.2 Prompt conversation à rôles multiples (system, assistant, human)

Ce type de prompt utilise la fonction `.from_messages( )` et permet de définir **plusieurs messages avec des rôles explicites** (system, human, etc.).
C’est un prompt structuré, idéal pour guider précisément le comportement du modèle dans un contexte conversationnel.

In [13]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    # Message système : définit le rôle et le comportement global du modèle
    ("system", "Tu es un expert en mathématiques et un pédagogue dans ce domaine."),
    # Message utilisateur : pose une question contenant deux variables
    ("human", "Calcule le double de {value_1}, puis celui de {value_2}")
])

# Injection des valeurs dans les variables du prompt
prompt_value = chat_prompt_template.invoke({"value_1": 12, "value_2": 34})

# Envoi du prompt structuré au modèle pour obtenir une réponse
result = model.invoke(prompt_value)

display(Markdown(result.content))

Voici les calculs étape par étape :

1.  **Le double de 12 :**
    Pour trouver le double d'un nombre, il faut le multiplier par 2 (ou l'ajouter à lui-même).
    $$12 \times 2 = 24$$

2.  **Le double de 34 :**
    De la même manière, nous multiplions 34 par 2.
    $$34 \times 2 = 68$$

**Résultats :**
Le double de 12 est **24**.
Le double de 34 est **68**.

### 🧩 Exercices

#### Exercice 5

Construire un assistant capable d’adopter le style d’un philosophe célèbre pour répondre à des questions existentielles.
1.	Créez un ChatPromptTemplate avec :
- un message system définissant l’IA comme un philosophe précis ({philosopher}),
- un message human contenant une question {question}.
2.	Injectez des variables avec :
- un nom de philosophe (ex. : Socrate, Nietzsche, Simone de Beauvoir),
- une question philosophique.
3.	Affichez la réponse du modèle, en observant si le style correspond au philosophe choisi.

In [14]:
# Votre code ici

#### Exercice 6

Simulez une conversation entre un utilisateur et un LLM autour d’un sujet (ex. : mathématiques, littérature, programmation) en construisant dynamiquement le prompt avec `ChatPromptTemplate`.

Implémentez une boucle qui :
- Initialise un prompt avec un message system.

À chaque tour :
- Prend une entrée utilisateur (input()),
- Ajoute un message human,
- Envoie le tout au LLM,
- Affiche la réponse,
- Ajoute un message de type/role `assistant` contenant la réponse (💡 ce rôle correspond à la réponse de l’IA et est l'équivalent de AIMessage).
- Arrête la conversation si l’utilisateur entre “stop”.

In [15]:
# Votre code ici